In [23]:
import cv2
import os
import pickle
import mediapipe as mp
from tqdm import tqdm
from IPython.display import clear_output

In [ ]:
# 데이터셋 경로
DATASET_DIR = '/home/shin/deeplearning-repo-1/dataset/pushup'

# 좌표 추출 운동 설정
exericse_name = "stand_knee_raise"

pickle_name = exericse_name + ".pkl"

exericse_type = ["squat", "pushup", "stand_knee_raise"]
exericse_label = exericse_type.index(exericse_name)

# 시퀀스 사이즈
SEQUENCE_LENGTH = 20

# mediapipe 선언
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

sequences = []
landmark_points = [ 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28 ]

with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5) as pose:
    clear_output(wait=True)
    for video_file in tqdm(os.listdir(DATASET_DIR)):
        video_path = os.path.join(DATASET_DIR, video_file)
        cap = cv2.VideoCapture(video_path)
        frames = []
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(frame_rgb)
            if results.pose_landmarks:
                landmarks = []

                lm = results.pose_landmarks.landmark

                # 기준점
                neck_x = (lm[11].x + lm[12].x) / 2
                neck_y = (lm[11].y + lm[12].y) / 2
                for idx in landmark_points:
                    x = lm[idx].x - neck_x
                    y = lm[idx].y - neck_y
                    landmarks.extend([x, y])
                frames.append(landmarks)
                if len(frames) >= SEQUENCE_LENGTH:
                    sequences.append(frames[-SEQUENCE_LENGTH:])
        cap.release()

  0%|          | 0/50 [00:00<?, ?it/s]W0000 00:00:1742548569.617550  300205 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742548569.640763  300215 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742548569.651917  300203 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742548569.679333  300214 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
100%|██████████| 50/50 [01:38<00:00,  1.97s/it]


In [22]:
with open(pickle_name, 'wb') as f:
    pickle.dump(sequences, f)

    print(exericse_name + " 데이터 저장 완료!")

squat 데이터 저장 완료!
